# Imports

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px 
import seaborn as sns

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

# Weather Data

In [20]:
weather_data = pd.read_csv('NYweather.csv', parse_dates=['datetime'])


In [21]:
crime_data = pd.read_csv('NYPD_Complaint_Data_Current.csv')
weather_data = pd.read_csv('NYweather.csv')

# Convert 'CMPLNT_FR_DT' in crime_data to datetime format with error handling
crime_data['CMPLNT_FR_DT'] = pd.to_datetime(crime_data['CMPLNT_FR_DT'], format='%m/%d/%Y', errors='coerce').dt.date

# Drop rows with invalid 'CMPLNT_FR_DT'
crime_data = crime_data.dropna(subset=['CMPLNT_FR_DT'])

# Filter between 2024-01-01 and 2024-10-01
start_date = pd.to_datetime('2024-01-01').date()
end_date = pd.to_datetime('2024-10-01').date()
crime_data = crime_data[(crime_data['CMPLNT_FR_DT'] >= start_date) & (crime_data['CMPLNT_FR_DT'] <= end_date)]

# Convert 'datetime' in weather_data to datetime format
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'], format='%Y-%m-%d').dt.date

# Merge crime_data with weather_data on the date
crime_weather_data = pd.merge(crime_data, weather_data, how='left', left_on='CMPLNT_FR_DT', right_on='datetime')

# Drop unnecessary 'datetime' column from weather_data
crime_weather_data = crime_weather_data.drop(columns=['datetime'])

# Save the combined data
crime_weather_data.to_csv('Crime_Weather_Combined.csv', index=False)

crime_weather_data.head()

/var/folders/rv/xy288w3d3sz5n6227tysh3nr0000gn/T/ipykernel_3545/337948210.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_data = pd.read_csv('NYPD_Complaint_Data_Current.csv')


,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column,name,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,280510523,14,MANHATTAN,2024-01-11,04:00:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,104,FELONY,INSIDE,RAPE,(null),PATROL BORO MAN SOUTH,157.0,RAPE 1,HOTEL/MOTEL,01/14/2024,(null),18-24,BLACK,U,NaN,18-24,BLACK,F,985764.0,213806.0,40.753533,-73.994537,"(40.7535327012632, -73.9945368920152)",POINT (-73.9945368920152 40.7535327012632),newyork,25.0,16.0,18.9,25.0,16.0,18.9,15.5,81.9,0.0,0,0.0,NaN,0,0,16.6,13.0,59.3,1018.6,21.1,4.3,197.7,17.0,7,10,2024-01-11T07:08:58,2024-01-11T17:59:45,0.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"42752099999,42662099999,42754099999,remote,VAID"
1,289019347,1,MANHATTAN,2024-06-20,16:12:00,06/20/2024,16:13:00,COMPLETED,(null),NaN,1,N.Y. TRANSIT POLICE,578,VIOLATION,(null),HARRASSMENT 2,(null),PATROL BORO MAN SOUTH,637.0,"HARASSMENT,SUBD 1,CIVILIAN",TRANSIT - NYC SUBWAY,06/20/2024,FULTON STREET,45-64,BLACK,M,2.0,25-44,BLACK,F,982466.0,197787.0,40.709556,-74.006435,"(40.70955616028041, -74.00643483276845)",POINT (-74.00643483276845 40.70955616028041),newyork,35.0,24.0,28.9,39.1,24.0,31.2,21.9,67.4,0.0,0,0.0,rain,0,0,36.4,24.1,247.8,1002.7,42.2,5.7,212.8,18.2,9,30,2024-06-20T05:42:39,2024-06-20T19:14:00,0.44,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"42752099999,42662099999,42754099999,VAID"
2,283688417,121,STATEN ISLAND,2024-03-11,10:00:00,03/13/2024,13:20:00,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,341,MISDEMEANOR,FRONT OF,PETIT LARCENY,(null),PATROL BORO STATEN ISLAND,339.0,"LARCENY,PETIT FROM OPEN AREAS,",MAILBOX OUTSIDE,03/13/2024,(null),(null),(null),(null),NaN,25-44,BLACK,F,938414.0,170500.0,40.634542,-74.165142,"(40.634542, -74.165142)",POINT (-74.165142 40.634542),newyork,33.2,15.0,25.2,32.0,15.0,24.7,9.8,39.6,0.0,0,0.0,NaN,0,0,22.7,14.8,299.9,1017.1,0.0,6.4,279.1,24.0,9,10,2024-03-11T06:38:56,2024-03-11T18:34:31,0.03,Clear,Clear conditions throughout the day.,clear-day,"42752099999,42662099999,42754099999,remote,VAID"
3,281033034,72,BROOKLYN,2024-01-23,19:00:00,01/24/2024,07:15:00,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,121,FELONY,(null),CRIMINAL MISCHIEF & RELATED OF,(null),PATROL BORO BKLYN SOUTH,267.0,"MISCHIEF, CRIMINAL 3 & 2, OF M",STREET,01/24/2024,(null),UNKNOWN,UNKNOWN,U,NaN,25-44,WHITE,F,983763.0,179737.0,40.660013,-74.001755,"(40.660013201115774, -74.00175528932395)",POINT (-74.00175528932395 40.660013201115774),newyork,23.2,8.0,16.3,23.2,8.0,16.3,4.7,48.4,0.0,0,0.0,NaN,0,0,19.8,22.3,41.7,1018.1,13.5,6.7,221.0,19.1,8,10,2024-01-23T07:08:36,2024-01-23T18:08:13,0.41,Clear,Clear conditions throughout the day.,clear-day,"42752099999,42662099999,42754099999,remote,VAID"
4,288178776,68,BROOKLYN,2024-05-28,10:00:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,341,MISDEMEANOR,(null),PETIT LARCENY,(null),PATROL BORO BKLYN SOUTH,353.0,"LARCENY,PETIT FROM BUILDING,UNATTENDED, PACKAG...",RESIDENCE-HOUSE,06/05/2024,(null),(null),(null),(null),NaN,25-44,ASIAN / PACIFIC ISLANDER,M,975766.0,165541.0,40.621044,-74.030559,"(40.621044, -74.030559)",POINT (-74.030559 40.621044),newyork,40.3,27.0,33.4,43.7,29.2,36.6,20.9,50.3,0.0,0,0.0,NaN,0,0,67.0,38.0,274.2,1001.4,9.3,6.4,334.5,29.0,10,10,2024-05-28T05:42:13,2024-05-28T19:05:57,0.67,Clear,Clear conditions throughout the day.,clear-day,"42752099999,42662099999,427540